In [2]:
from app.utils import create_pad_sequences, clean_text, load_model, load_tokenizer, BODY_COL_NAME
from typing import Dict, Any
from deep_translator import GoogleTranslator
import pandas as pd

ROOT_DIR = '../../'

In [3]:
model_by_name: Dict[str, Dict[str, Any]] = {
    'fishing': {
        'threshold': 0.985,
    },
    'fraud': {
        'threshold': 0.90,
    },
    'spam': {
        'threshold': 0.98,
    },
}

for name in model_by_name.keys():
    model_by_name[name]['model'] = load_model(f'{ROOT_DIR}models/{name}')
    model_by_name[name]['tokenizer'] = load_tokenizer(f'{ROOT_DIR}tokenizers/{name}')

In [4]:
def predict(message: str):
    message = clean_text(message)
    df = pd.DataFrame([message], columns=[BODY_COL_NAME])

    for name, data in model_by_name.items():
        tokenizer = data['tokenizer']
        model = data['model']
        message_pad = create_pad_sequences(tokenizer, df[BODY_COL_NAME])
        prediction = model.predict(message_pad)
        prediction = prediction[0][0]
        print(name, prediction, f'Trigger: {prediction > data["threshold"]}')

In [5]:
import sys

message = input("Enter eMAIL message: ")

if not message:
    sys.exit(1)

translator = GoogleTranslator(source='auto', target='en')
message = translator.translate(message)
print(message)
predict(message)